In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/heart-disease-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/heart-disease-dataset


In [2]:
import os

print("Dataset files:", os.listdir(path))

Dataset files: ['heart.csv']


In [3]:
import numpy as np
import pandas as pd
csv = os.path.join(path, "heart.csv")

dataset = pd.read_csv(csv)
print(dataset.head(10))

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   
5   57    1   0       140   192    0        1      148      0      0.4      1   
6   56    0   1       140   294    0        0      153      0      1.3      1   
7   44    1   1       120   263    0        1      173      0      0.0      2   
8   52    1   2       172   199    1        1      162      0      0.5      2   
9   57    1   2       150   168    0        1      174      0      1.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1 

In [4]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Missing Data

In [5]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, :14])
X[:, :14] = imputer.transform(X[:, :14])

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=0)

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X_train[:, :14] = scaler.fit_transform(X_train[:,:14])
X_test[:, :14] = scaler.fit_transform(X_test[:,:14])

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
input_dim = 13
hidden_units = 15
num_hidden_layers = 7
output_dim = 1
learning_rate = 0.001
activation = 'relu'
output_activation = 'sigmoid'

with tf.device('/device:GPU:0'):
  model = Sequential()

# Input layer
  model.add(Dense(hidden_units, input_dim=input_dim, activation=activation))

# Hidden layers
  for _ in range(num_hidden_layers - 1):
      model.add(Dense(hidden_units, activation=activation))
      model.add(Dropout(0.2)) #regularization

# Output layer
  model.add(Dense(output_dim, activation=output_activation))

# Compile model
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(
      loss='binary_crossentropy',  #NLL
      optimizer=optimizer,
      metrics=['binary_accuracy']
  )

# Model Train
  history = model.fit(
      X_train,
      y_train,
      epochs=1000,
      batch_size=32,
      validation_split=0.1,
      verbose=2
  )

# Evaluate model
  results = model.evaluate(X_test, y_test)
  print(f"Loss on test set: {results[0]:.4f}")
  print(f"Accuracy on test set: {results[1]:.4f}")

Epoch 1/1000
8/8 - 10s - 1s/step - binary_accuracy: 0.5738 - loss: 0.6900 - val_binary_accuracy: 0.3929 - val_loss: 0.7057
Epoch 2/1000
8/8 - 4s - 459ms/step - binary_accuracy: 0.5861 - loss: 0.6854 - val_binary_accuracy: 0.3929 - val_loss: 0.7085
Epoch 3/1000
8/8 - 0s - 37ms/step - binary_accuracy: 0.5656 - loss: 0.6809 - val_binary_accuracy: 0.3929 - val_loss: 0.7095
Epoch 4/1000
8/8 - 0s - 36ms/step - binary_accuracy: 0.5738 - loss: 0.6892 - val_binary_accuracy: 0.3929 - val_loss: 0.7086
Epoch 5/1000
8/8 - 0s - 35ms/step - binary_accuracy: 0.5697 - loss: 0.6879 - val_binary_accuracy: 0.3929 - val_loss: 0.7058
Epoch 6/1000
8/8 - 0s - 18ms/step - binary_accuracy: 0.5697 - loss: 0.6712 - val_binary_accuracy: 0.3929 - val_loss: 0.7041
Epoch 7/1000
8/8 - 0s - 15ms/step - binary_accuracy: 0.5615 - loss: 0.6762 - val_binary_accuracy: 0.3929 - val_loss: 0.7034
Epoch 8/1000
8/8 - 0s - 11ms/step - binary_accuracy: 0.5738 - loss: 0.6792 - val_binary_accuracy: 0.3929 - val_loss: 0.6995
Epoch 9/